In [6]:
!sudo apt update
!sudo apt -y install build-essential
!pip install networkx node2vec python-Levenshtein sklearn

Hit:1 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal-backports InRelease



25 packages can be upgraded. Run 'apt list --upgradable' to see them.



build-essential is already the newest version (12.8ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


In [7]:
import pandas as pd

index = pd.read_csv('https://ameo.dev/spotify_items.csv.gz', compression = 'gzip')

In [8]:
index.head()

,id,spotify_id
0,146079057,0000BlyH0BpSivfg8y7QSb
1,146505418,0001cekkfdEBoMlwVQvpLg
2,88525725,0001wHqxbF2YYRQxGdbyER
3,116519192,0001ZVMPt41Vwzt1zsmuzp
4,115213790,0002XY9y3JhjzTZqNCqEcv


In [9]:
spotify_ids_by_id = {}

for (id, artist_spotify_id) in index.itertuples(index=False):
  spotify_ids_by_id[int(id)] = artist_spotify_id

In [10]:
ids_by_spotify_id = {}

for (id, spotify_id) in spotify_ids_by_id.items():
  ids_by_spotify_id[spotify_id] = id

In [11]:
related_artists_raw = pd.read_csv('https://ameo.dev/related_artists.csv.gz', compression = 'gzip')

In [12]:
related_artists_raw.head()

,artist_spotify_id,related_artists_json
0,1,"[""5uCktjKM1Rf9VnUvFD6j17"",""5FAjeD9eW52qvpik9Os..."
1,2,"[""3j24yoSFDD11h1OszJFfVY"",""2OgP0RGmDrXN9kOWRNf..."
2,3,"[""0UM3yh4PeDSWa9cnHJ0jgW"",""0q0K0FV5t8OnVpbIQTX..."
3,4,"[""7gjAu1qr5C2grXeQFFOGeh"",""1qcU8NabUQ5efQf8UGW..."
4,5,"[""3QpNnKSjUIIMwFei8h3eOj"",""04Y5zXsFqYRzDr6tMXj..."


In [13]:
related_artists_raw['id'] = related_artists_raw['artist_spotify_id']
related_artists_raw['artist_spotify_id'] = related_artists_raw['artist_spotify_id'].apply(lambda id: spotify_ids_by_id[int(id)])

In [14]:
related_artists_raw.head()

,artist_spotify_id,related_artists_json,id
0,4IuaUq1boR1KVcutaID4Zx,"[""5uCktjKM1Rf9VnUvFD6j17"",""5FAjeD9eW52qvpik9Os...",1
1,2UwJRAgSOi1zcLkvUNc8XL,"[""3j24yoSFDD11h1OszJFfVY"",""2OgP0RGmDrXN9kOWRNf...",2
2,4BrtGOYp8sytG9MV91DROT,"[""0UM3yh4PeDSWa9cnHJ0jgW"",""0q0K0FV5t8OnVpbIQTX...",3
3,1xU878Z1QtBldR7ru9owdU,"[""7gjAu1qr5C2grXeQFFOGeh"",""1qcU8NabUQ5efQf8UGW...",4
4,2Ub06wAIR1hERODcCkKhzx,"[""3QpNnKSjUIIMwFei8h3eOj"",""04Y5zXsFqYRzDr6tMXj...",5


In [15]:
import json

related_artists_raw['related_artists'] = related_artists_raw['related_artists_json'].apply(lambda x: json.loads(x))

In [16]:
del related_artists_raw['related_artists_json']
related_artists_raw.head()

,artist_spotify_id,id,related_artists
0,4IuaUq1boR1KVcutaID4Zx,1,"[5uCktjKM1Rf9VnUvFD6j17, 5FAjeD9eW52qvpik9OsnV..."
1,2UwJRAgSOi1zcLkvUNc8XL,2,"[3j24yoSFDD11h1OszJFfVY, 2OgP0RGmDrXN9kOWRNfEu..."
2,4BrtGOYp8sytG9MV91DROT,3,"[0UM3yh4PeDSWa9cnHJ0jgW, 0q0K0FV5t8OnVpbIQTXOh..."
3,1xU878Z1QtBldR7ru9owdU,4,"[7gjAu1qr5C2grXeQFFOGeh, 1qcU8NabUQ5efQf8UGW6C..."
4,2Ub06wAIR1hERODcCkKhzx,5,"[3QpNnKSjUIIMwFei8h3eOj, 04Y5zXsFqYRzDr6tMXj6a..."


In [17]:
import networkx as nx
graph = nx.Graph()

all_related_artists = related_artists_raw[:100_000]

graph.add_nodes_from(all_related_artists['id'])

In [18]:
for (artist_spotify_id, id, related_artists) in all_related_artists.itertuples(index=False):
  for related_artist_spotify_id in related_artists:
    related_artist_id = ids_by_spotify_id.get(related_artist_spotify_id)
    if related_artist_id is None:
      continue

    graph.add_edge(id, related_artist_id)
    graph.add_edge(related_artist_id, id)

In [19]:
from node2vec import Node2Vec

node2vec = Node2Vec(graph, dimensions=3, walk_length=40, num_walks=15, workers=14)

Computing transition probabilities:   0%|          | 0/354245 [00:00<?, ?it/s]

Generating walks (CPU: 6):   0%|          | 0/1 [00:00<?, ?it/s]/opt/conda/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
Generating walks (CPU: 14): 100%|██████████| 1/1 [05:57<00:00, 357.31s/it]


In [20]:
model = node2vec.fit(window=10, min_count=1, workers=14, hs=0, negative=10)

In [25]:
model.wv.get_vector('50')

array([ 1.353515 ,  0.9039356, -2.7190468], dtype=float32)

In [26]:
model.wv.n_similarity(['50'], ['4'])

0.79200864

In [27]:
tv_room_id = str(ids_by_spotify_id['0EFfq11AFfQOjJ0PvAvvSW'])
sleigh_bells_id = str(ids_by_spotify_id['59pWgeY26Q6yJy37QvJflh'])
red_velvet_id = str(ids_by_spotify_id['1z4g3DjTBBZKhvAroFlhOM'])
siinamota_id = str(ids_by_spotify_id['4qfQTMwNNuBKJ4rF9Lo4bD'])
current_value_id = str(ids_by_spotify_id['6B9SjvZNSQZkeJDH17oBSO'])
shiiit_id = str(ids_by_spotify_id['0RfocEzLe78RexLTeU1K2p'])
skrillex_id = str(ids_by_spotify_id['5he5w2lnU9x7JFhnwcekXX'])
kobaryo_id = str(ids_by_spotify_id['1Y81Ch90opScfpMfN17lZb'])
loona_id = str(ids_by_spotify_id['52zMTJCKluDlFwMQWmccY7'])
skylar_spence_id = str(ids_by_spotify_id['0x0u0jCVf5Jf4DNh45XPXL'])
taylor_swift_id = str(ids_by_spotify_id['06HL4z0CvFAxyc27GXpf02'])
freddie_dredd_id = str(ids_by_spotify_id['0dlDsD7y6ccmDm8tuWCU6F'])
g_herbo_id = str(ids_by_spotify_id['5QdEbQJ3ylBnc3gsIASAT5'])
fox_stevenson_id = str(ids_by_spotify_id['2BQWHuvxG4kMYnfghdaCIy'])
lorde_id = str(ids_by_spotify_id['163tK9Wjr9P9DmM0AVK7lm'])
neon_indian_id = str(ids_by_spotify_id['0KydPQPUUoTNhmiHKOg5Er'])
quok_id = str(ids_by_spotify_id['1hjJ4bsbNMmbIih3d9Pq1H'])
strfkr_id = str(ids_by_spotify_id['2Tz1DTzVJ5Gyh8ZwVr6ekU'])
beatles_id = str(ids_by_spotify_id['3WrFJ7ztbogyGnTHbHJFl2'])
drake_id = str(ids_by_spotify_id['3TVXtAsR1Inumwj472S9r4'])
anamanaguchi_id = str(ids_by_spotify_id['2UwJRAgSOi1zcLkvUNc8XL'])
ed_sheeran_id = str(ids_by_spotify_id['6eUKZXaKkcviH0Ku9w2n3V'])
frank_sinatra_id = str(ids_by_spotify_id['1Mxqyy3pSjf8kZZL4QVxS0'])
tame_impala_id = str(ids_by_spotify_id['5INjqkS1o8h1imAzPqGZBb'])
queen_id = str(ids_by_spotify_id['1dfeR4HaWDbWqFHLkxsg1d'])
ngutp_id = str(ids_by_spotify_id['0j5kVHxvTgUN4nBIPKCLRJ'])
infected_mushroom_id = str(ids_by_spotify_id['6S2tas4z6DyIklBajDqJxI'])
dither_id = str(ids_by_spotify_id['2CTPZndAY5BwdgHk8AX0B7'])
john_williams_id = str(ids_by_spotify_id['3dRfiJ2650SZu6GbydcHNb'])
dababy_id = str(ids_by_spotify_id['4r63FhuTkUYltbVAg5TQnk'])
blackbear_id = str(ids_by_spotify_id['2cFrymmkijnjDg9SS92EPM'])
chon_id = str(ids_by_spotify_id['2JFljHPanIjYy2QqfNYvC0'])
deco_id = str(ids_by_spotify_id['7kZTWx6cRLc0TSRPq1XBMP'])
dareharu_id = str(ids_by_spotify_id['4mcnvjLyW44CoKXDkUjPzh'])
kupla_id = str(ids_by_spotify_id['7daSp9zXk1dmqNxwKFkL35'])
pinnocchiop_id = str(ids_by_spotify_id['3b7jPCedJ2VH4l4rcOTvNC'])
sugar_and_co_id = str(ids_by_spotify_id['4RCVsWUvlaucR1HTy8Tk7a'])

(tv_room_id, sleigh_bells_id, red_velvet_id, siinamota_id, current_value_id, shiiit_id, skrillex_id, kobaryo_id, loona_id)

('912', '65', '643', '7801598', '57179651', '9318669', '248', '1339641', '515')

In [29]:
a = quok_id
b = dareharu_id
diff = model.wv.get_vector(a) + model.wv.get_vector(b)*0.3
diff 

closest = model.wv.similar_by_vector(diff)

def compute_res(close):
    a_dist = model.wv.distance(close, a)
    b_dist = model.wv.distance(close, b)

    return {'spotify_id': spotify_ids_by_id[int(close)],
            'a_dist': a_dist,
            'b_dist': b_dist,
            'abs_dst_diff': abs(a_dist - b_dist)
           }

print("distance: ", model.wv.distance(a, b))

[compute_res(close[0]) for close in closest]

distance:  1.0152859389781952


[{'spotify_id': '6ReMNCOSnarYESHJpHbe7m',
  'a_dist': 0.3004125952720642,
  'b_dist': 0.2962321639060974,
  'abs_dst_diff': 0.004180431365966797},
 {'spotify_id': '2gzcB4vMIStEDZiQlvywGm',
  'a_dist': 0.2930290699005127,
  'b_dist': 0.30364739894866943,
  'abs_dst_diff': 0.010618329048156738},
 {'spotify_id': '27YmfBn7iiVbJulvIm03nQ',
  'a_dist': 0.2927485704421997,
  'b_dist': 0.30393218994140625,
  'abs_dst_diff': 0.011183619499206543},
 {'spotify_id': '1d1O2KbOZhJWMEDgr0yXaC',
  'a_dist': 0.30284202098846436,
  'b_dist': 0.29383349418640137,
  'abs_dst_diff': 0.009008526802062988},
 {'spotify_id': '5URaqWGLWi0maxmjctj8I0',
  'a_dist': 0.29629331827163696,
  'b_dist': 0.30039137601852417,
  'abs_dst_diff': 0.004098057746887207},
 {'spotify_id': '4c1JXtOtdiAGTdA5zGfjc2',
  'a_dist': 0.2939690947532654,
  'b_dist': 0.30273741483688354,
  'abs_dst_diff': 0.008768320083618164},
 {'spotify_id': '7mKZJ0SITREZQovQF6Qeb4',
  'a_dist': 0.29810261726379395,
  'b_dist': 0.2986191511154175,
  'a

In [30]:
furthest = model.wv.similar_by_vector(-model.wv.get_vector(drake_id))
[compute_res(far[0]) for far in furthest]

[{'spotify_id': '5hTeASXqJCdxbg5Ow1ZS6H',
  'a_dist': 1.667548954486847,
  'b_dist': 1.3572876155376434,
  'abs_dst_diff': 0.3102613389492035},
 {'spotify_id': '601KLl5xZbrE3tcfUP39lZ',
  'a_dist': 1.6710955500602722,
  'b_dist': 1.3869051933288574,
  'abs_dst_diff': 0.2841903567314148},
 {'spotify_id': '4UKPMYDWKr8KsQLibePkc5',
  'a_dist': 1.6844033002853394,
  'b_dist': 1.3865554928779602,
  'abs_dst_diff': 0.29784780740737915},
 {'spotify_id': '6U5rJoNT74QPyHfnu7Ho1I',
  'a_dist': 1.6305699348449707,
  'b_dist': 1.417829304933548,
  'abs_dst_diff': 0.21274062991142273},
 {'spotify_id': '5MzBifGDrfNPrR0y7q1k9D',
  'a_dist': 1.7151669263839722,
  'b_dist': 1.36569544672966,
  'abs_dst_diff': 0.34947147965431213},
 {'spotify_id': '1i1I7TZwUhyMaM6xN7Q1PG',
  'a_dist': 1.6689512729644775,
  'b_dist': 1.2691713869571686,
  'abs_dst_diff': 0.39977988600730896},
 {'spotify_id': '2k9u6sq9kF1Oxn8NwdPWdL',
  'a_dist': 1.7236366271972656,
  'b_dist': 1.288521647453308,
  'abs_dst_diff': 0.43511

In [31]:
spotify_ids_by_id[int(furthest[0][0])]
# all_related_artists
# spotify_ids_by_id[red_velvet_id]

'5hTeASXqJCdxbg5Ow1ZS6H'

In [32]:
def get_most_similar_to_both(id_a, id_b):
    best_distance = 1000000
    best_id = None
    middle = model.wv.get_vector(id_a) + model.wv.get_vector(id_b)
    for id in all_related_artists['id']:
        id = str(id)
        if id == id_a or id == id_b:
            continue
        a_dst = model.wv.distance(id_a, id)
        b_dst = model.wv.distance(id_b, id)
        cost = (a_dst**2 + b_dst**2)
        dist_diff = abs(a_dst - b_dst)
        distance = model.wv.distances(middle, [id])[0]
        if distance < best_distance:
            best_distance = distance
            best_id = id
        
    return best_id

In [33]:
get_most_similar_to_both(loona_id, skylar_spence_id)

'108305'

In [34]:
spotify_ids_by_id[68670620]

'1vpHPL6nejNAg9GXYrJ5ck'

In [35]:
model.wv.distance('68670620', loona_id)

0.00509190559387207

In [36]:
model.wv.distance('68670620', skylar_spence_id)

0.7759359031915665

In [37]:
model.wv.save_word2vec_format("50k_corpus_3_dims_new_params.w2v", "50k_corpus_3_dims_new_params.vocab")

In [38]:
# (model.wv.get_vector(dareharu_id) ,model.wv.get_vector(deco_id))

def get_val_for_dim(id, dim_ix):
    id = str(id)
    vec = model.wv.get_vector(id)
    return vec[dim_ix]

def get_extremes_for_dim(dim_ix, buckets, count):
    vals = [(spotify_ids_by_id[id], get_val_for_dim(id, dim_ix)) for id in all_related_artists['id']]
    
    vals.sort(key = lambda x: x[1])
    partials_count = buckets - 2
    partial_stride = int(len(vals) / partials_count)
    
    out = []
    out.append(vals[:count])
    
    for partial_ix in range(1, partials_count):
        middle = partial_ix * partial_stride
        out.append(vals[(middle - int(count / 2)):(middle + int(count / 2))])
    
    out.append(vals[-count:])
            
    return out

In [39]:
get_extremes_for_dim(0, 20, 4)

[[('2ovJLSejirvJybNers2YG6', -12.486315),
  ('50BzmPPHNTQjeLW5JvrZJs', -12.470005),
  ('62ABL5pqowZJin1fkheWo7', -12.30272),
  ('4z6dlDfmoZstfKLVC60nM0', -12.292496)],
 [('2tR9SGGAyNGg8A3GBoOFbP', -9.395903),
  ('1OY65G9NJJHsSt1lGLZU6q', -9.395884),
  ('3zrnHI8M8fvS6VmPF8behc', -9.395685),
  ('3AGrDAZx8XN9LhuVxwVN26', -9.3956375)],
 [('30TUV9gTBJ8ffWRMe3O2J0', -7.9888005),
  ('3lSs59gV3uuobbJ6uOtizq', -7.9886775),
  ('6nu39YK7u99qv6JdygWuxG', -7.9886584),
  ('6uqOd0qw9avpT0vCyCZ4DX', -7.9876323)],
 [('13BVQ7HFExOqU7IIGLj5zz', -6.969541),
  ('566KzI0kXa1dQ3em27iQtr', -6.969527),
  ('5RnsHSCPrlCxP9A1NTT7jS', -6.968781),
  ('37bsUoMW0Hbqj1l0MxDkDw', -6.9682755)],
 [('3DDcmHnwKjM4sQ8qm34JMU', -5.623506),
  ('3PS5OcUQlmA1epxoBau3YI', -5.6231227),
  ('5eRKMbpTG8cIUdfomXcJ80', -5.622978),
  ('3VNITwohbvU5Wuy5PC6dsI', -5.6226377)],
 [('0EziqO4Mehje1x6hA1Fg2m', -4.0594497),
  ('6nnspeopmJAG07xOxHmqTu', -4.0592394),
  ('68Uwx1d3EbjDJhYsLjqvIq', -4.059202),
  ('08GQAI4eElDnROBrJRGE0X', -4.0591693